In [11]:
import torch
import numpy as np
from mmaction.apis import init_recognizer, inference_recognizer, train_model
from mmaction.datasets import RawframeDataset
import utils

## Construct Training Dataset and Pipeline

In [5]:
DATASET_PATH = '/vision/group/ntu-rgbd/'

### Setup data config

In [37]:
# Using training pipeline from the Swin Tiny Config file set above
train_ann_filename = DATASET_PATH + 'rgb_train_ann.txt'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
train_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=32, frame_interval=2, num_clips=1),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, 256)),
    dict(type='RandomResizedCrop'),
    dict(type='Resize', scale=(224, 224), keep_ratio=False),
    dict(type='Flip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs', 'label'])
]

val_pipeline = [
    dict(type='DecordInit'),
    dict(
        type='SampleFrames',
        clip_len=32,
        frame_interval=2,
        num_clips=1,
        test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, 256)),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Flip', flip_ratio=0),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]

test_pipeline = [
    dict(type='DecordInit'),
    dict(
        type='SampleFrames',
        clip_len=32,
        frame_interval=2,
        num_clips=4,
        test_mode=True),
    dict(type='DecordDecode'),
    dict(type='Resize', scale=(-1, 224)),
    dict(type='ThreeCrop', crop_size=224),
    dict(type='Flip', flip_ratio=0),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='FormatShape', input_format='NCTHW'),
    dict(type='Collect', keys=['imgs', 'label'], meta_keys=[]),
    dict(type='ToTensor', keys=['imgs'])
]
num_classes = 60 # Need to change to 120 for NTU RGB-D 120
train_dataset = RawframeDataset(train_ann_filename, train_pipeline, num_classes=60)

In [40]:
dataset_type = 'RawframeDataset'
ann_file_train = DATASET_PATH + 'rgb_train_ann.txt'
ann_file_val = DATASET_PATH + 'rgb_valid_ann.txt'
ann_file_test = DATASET_PATH + 'rgb_test_ann.txt'

data = dict(
    videos_per_gpu=8,
    workers_per_gpu=4,
    val_dataloader=dict(
        videos_per_gpu=1,
        workers_per_gpu=1
    ),
    test_dataloader=dict(
        videos_per_gpu=1,
        workers_per_gpu=1
    ),
    train=dict(
        type=dataset_type,
        ann_file=ann_file_train,
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=ann_file_val,
        pipeline=val_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=ann_file_test,
        pipeline=test_pipeline))
evaluation = dict(
    interval=5, metrics=['top_k_accuracy', 'mean_class_accuracy'])

## Load pre-trained weights and change head

In [28]:
# Load Tiny Video-Swin Transformer pre-trained on Kinetics
config_file = '../Video-Swin-Transformer/configs/recognition/swin/swin_tiny_patch244_window877_kinetics400_1k.py'
checkpoint_file = '../models/pre-trained/swin/swin_tiny_patch244_window877_kinetics400_1k.pth'
device = 'cpu' #'cuda:0' # or 'cpu' # CHANGE WHEN USING GPU/CPU
device = torch.device(device)
model = init_recognizer(config_file, checkpoint_file, device=device)

load checkpoint from local path: ../models/pre-trained/swin/swin_tiny_patch244_window877_kinetics400_1k.pth


In [29]:
# Change the head of the model
# Uses He initialization, could be something to look into: 
# https://arxiv.org/pdf/2002.06305.pdf investigates how different seeds and data ordering
# strongly changes performance of the trained model
model.cls_head.fc_cls = torch.nn.Linear(in_features=768, out_features=num_classes, bias=True)

## Freeze model weights (except for head)
This tutorial talks about fine-tuning: https://mmaction2.readthedocs.io/en/latest/tutorials/2_finetune.html

In [34]:
for param in model.parameters():
    param.requires_grad = False
model.cls_head.fc_cls.weight.requires_grad = True

## Train model

In [41]:
train_model(model, train_dataset, data)

AttributeError: 'dict' object has no attribute 'log_level'

## Model training pipeline